<a href="https://colab.research.google.com/github/harry9879/Natural-language-processing-with-Naive-Bayes-and-Textvectorization/blob/main/natural_language_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-27 16:24:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-27 16:24:12 (75.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-27 16:24:13--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.145.128, 74.125.143.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-02-27 16:24:13 (133 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
unzip_data('nlp_getting_started.zip')

## Visualizing a text dataset

In [ ]:
import pandas as pd 
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac = 1 , random_state=2)
train_df_shuffled.head()

,id,keyword,location,text,target
3190,4579,emergency%20plan,North Hastings Ontario,Practice your families fire escape plan so eve...,0
6171,8801,sirens,"Nomad, USA",Fuck Sleeping With Sirens.,0
1196,1722,buildings%20burning,NaN,'i'm a Gemini' *children screaming buildings b...,1
680,982,blazing,"Pig Symbol, Alabama",Montgomery come for the blazing hot weather......,1
3358,4808,evacuated,NaN,I got evacuated from the cinema 30 mins throug...,0


In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
len(train_df) , len(test_df)

(7613, 3263)

In [ ]:
import random 
random_index = random.randint(0, len(train_df) - 5)

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorization = TextVectorization(max_tokens = None,
                                       standardize = 'lower_and_strip_punctuation',
                                       split='whitespace',
                                       output_mode='int',
                                       ngrams= None,
                                       output_sequence_length=None)

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences , valid_sentences , train_labels , val_labels= train_test_split(train_df['text'].to_numpy(), train_df['target'], test_size = 0.1)

In [ ]:
max_vocab_length = 10000
max_length = 15
text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       output_mode = 'int',
                                       output_sequence_length = max_length)

In [ ]:
text_vectorization.adapt(train_sentences)

In [ ]:
sample_sentence = 'There is a flood in my street'
text_vectorization([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 73,   9,   3, 236,   4,  13, 643,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence}\n \n\n vectorizatied version")
text_vectorization([random_sentence])

Original text: 
 def louis is tired plus the meltdown of fans when he confirmed he's a dad but some fans are reaching 
making conclusions and stuff
 

 vectorizatied version


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[5849, 1376,    9, 1448, 1815,    2,  481,    6, 1017,   46,   62,
         272,  268,    3, 1138]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorization.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words : {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words : ['paradise', 'paracord', 'para', 'papiichampoo', 'papicongress']


The parameters we care most about for our embedding layer: 
* `input_dim` = the size of our vocabulary
* `outpur_dim` = the size of the output embedding vector, for example, a value of 100 would mean token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim = 128, # output shape 
                             input_length = max_length # how long is each input
                             ) 
embedding

In [ ]:
random_sentence = random.choice(train_sentences)
print(f"original text: \n {random_sentence}\n\n embedded version")

sample_embed = embedding(text_vectorization([random_sentence]))
sample_embed

original text: 
 [GPV Wind] As of 06JST 6AUG: WNW 06JST 6AUG / E 12JST 6AUG / S 18JST 6AUG. http://t.co/l6jBjAj8dm

 embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04186611, -0.04342169, -0.03053417, ...,  0.01644851,
         -0.03100933,  0.03314732],
        [ 0.03430535,  0.00244356,  0.03914498, ...,  0.0226945 ,
          0.01806823, -0.03891955],
        [-0.00855809, -0.00013194, -0.03043452, ...,  0.01322384,
          0.02412455, -0.01268507],
        ...,
        [-0.04096977,  0.00941058, -0.02842174, ...,  0.0428638 ,
         -0.01125903,  0.00712656],
        [-0.04186611, -0.04342169, -0.03053417, ...,  0.01644851,
         -0.03100933,  0.03314732],
        [-0.02113135, -0.04557123, -0.04121157, ...,  0.04025618,
          0.03448508, -0.02513777]]], dtype=float32)>

In [ ]:
## Modelling a text dataset (running a series of experiments)

Now we've a got way to turn our text sequences into numbers,
it's time to start building a series of modelling experiments

we'll starts with a baseline and move on from there.

* Model 0 : Naive Bayes (Baseline)
* 